# Projecting Food Insecurity Rates in the US by County
## Feature Engineering
The following process imports a cleaned dataset produced from [cleaning_pt2.ipynb.](notebooks/cleaning_pt2) This notebook is used to produce new features that will be used in the modeling process.
### Flatiron School Data Science Capstone<br>By Khyatee Desai

In [145]:
import pandas as pd
import numpy as np
from itertools import combinations
import sklearn
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)
import pickle
import warnings
warnings.filterwarnings('ignore')

In [146]:
with open('../pickled/fully_cleaned_data.pickle', "rb") as input_file:
    df = pickle.load(input_file) 
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.0,94584.0,117854.0,81626.0,52774.0,431638.0,211570.0,81336.0,658834.0,14.8,2.48,87036.0,93630.0,259397.0,17519.0,39967.0,Alabama,Jefferson County,659972.0,312491.0,347481.0,355635.0,284082.0,2029.0,10378.0,347.0,24099.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.0,292505.0,19626.0,6.3
1,01117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,69723.0,22792.0,28911.0,35773.0,18511.0,159655.0,42429.0,19175.0,201168.0,6.2,2.65,23404.0,19762.0,74790.0,7624.0,4706.0,Alabama,Shelby County,206280.0,100304.0,105976.0,174094.0,24247.0,805.0,4403.0,101.0,11872.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0,107208.0,102400.0,4808.0,4.5
2,04003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,45974.0,20341.0,21109.0,12968.0,7566.0,76099.0,50498.0,14868.0,130807.0,13.1,2.47,23593.0,16328.0,48846.0,7947.0,7812.0,Arizona,Cochise County,127314.0,64661.0,62653.0,112238.0,5737.0,2165.0,2757.0,510.0,44374.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0,50969.0,46682.0,4287.0,8.4
3,04013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,53689.0,399455.0,593094.0,490927.0,273108.0,2444443.0,1204681.0,646167.0,3947382.0,12.7,2.74,507428.0,546028.0,1424244.0,373532.0,171581.0,Arizona,Maricopa County,4093648.0,2024659.0,2068989.0,3449404.0,235660.0,112383.0,172425.0,11190.0,1239835.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0,1961997.0,1848119.0,113878.0,5.8
4,04019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,46233.0,133694.0,149147.0,115392.0,81406.0,592298.0,364938.0,141211.0,993144.0,13.2,2.50,162075.0,149710.0,386155.0,69636.0,57099.0,Arizona,Pima County,1004229.0,494684.0,509545.0,858334.0,41043.0,42683.0,31905.0,2266.0,363063.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0,463126.0,435183.0,27943.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,49999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3560,51999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3561,53999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3562,54999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
df[df.Year=='2020']

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,01001,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga County, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,25485.100,24078.400,1406.700,5.5300
10,01003,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baldwin County, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,96586.200,90467.700,6118.500,6.3600
20,01005,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Barbour County, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8933.200,8290.600,642.600,7.1500
30,01007,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bibb County, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8685.600,8043.200,642.400,7.3400
40,01009,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Blount County, AL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,24704.900,23569.900,1135.000,4.6100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32452,72145,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vega Baja Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,12646.625,11223.375,1423.250,11.2375
32462,72147,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vieques Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2476.875,2252.000,224.875,9.0875
32472,72149,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Villalba Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6819.875,6013.500,806.375,11.6500
32482,72151,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yabucoa Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8106.625,7232.250,874.375,10.7500


### Create a percentage for each demographic

In [148]:
df.replace(-888888888.0,np.nan,inplace=True)

In [149]:
df['Percent_male'] = df['TOT_MALE']/df['TOT_POP']
df['Percent_female'] = df['TOT_FEMALE']/df['TOT_POP']
df['Percent_white'] = df['TOT_WHITE']/df['TOT_POP']
df['Percent_Black'] = df['TOT_BLACK']/df['TOT_POP']
df['Percent_native'] = df['TOT_NATIVE']/df['TOT_POP']
df['Percent_asian'] = df['TOT_ASIAN']/df['TOT_POP']
df['Percent_pacific'] = df['TOT_PACIFIC']/df['TOT_POP']
df['Percent_latinX'] = df['TOT_LATINX']/df['TOT_POP']

### Percentage PoC
Percentage of a county that is not white

In [150]:
df['Percent_PoC'] = 1-df['Percent_white']

### Workforce as a percentage of total population
Workforce represented as percentage, so that it can be compared across different counties

In [151]:
# percent of population that is working
df['Percent_working'] = df['Total_workforce']/df['TOT_POP']


### Sum of Food Establishments
Total number of food retail businesses, which is the sum of the three categories (wholesalers, restaraunts, and grocery stores)

In [152]:
df['Total_food_retail'] = df['Num_wholesale'].fillna(0)+ df['Num_restaraunts'].fillna(0)+df['Num_grocery'].fillna(0)

### Population divided by number of food establishments 
Looking at prevalence of food establishments as a function of population - aka how many Food Retail establishments exist per person within a County

In [153]:
df['Food_retail_per_person'] = df['Total_food_retail']/df['TOT_POP']


## ACS Survey Percentages
Take percentages of all features taken from the ACS survey, which use a slightly different (~5%) total population number for the denominator

In [154]:
df['Percent_disabled'] = df['pop_disabled']/df['TOT_POP']
df['Percent_hs_grad'] = df['pop_hs_grad']/df['TOT_POP']
df['Percent_bachelors'] = df['pop_bachelors']/df['TOT_POP']
df['Percent_grad_degree'] = df['pop_grad_degree']/df['TOT_POP']
df['Percent_priv_health'] = df['pop_priv_health']/df['TOT_POP']
df['Percent_public_health'] = df['pop_public_health']/df['TOT_POP']
df['Percent_no_health'] = df['pop_no_health']/df['TOT_POP']
df['Percent_65+'] = df['pop_65+']/df['TOT_POP']
df['Percent_non_citizen'] = df['pop_non_citizen']/df['TOT_POP']
df['Percent_hh_no_vehicle'] = df['hh_no_vehicle']/df['num_hh']
df['Percent_hh_SNAP'] = df['hh_SNAP']/df['num_hh']

### Drop raw count columns after deriving percentages

In [155]:
df.drop(['TOT_MALE','TOT_FEMALE','TOT_WHITE','TOT_BLACK','TOT_NATIVE','TOT_ASIAN','TOT_PACIFIC','TOT_LATINX',
         'pop_disabled', 'pop_hs_grad','pop_bachelors', 'pop_grad_degree', 'pop_priv_health', 'pop_public_health',
        'pop_no_health', 'pop_65+','pop_non_citizen','hh_no_vehicle','hh_SNAP','pop_total'],axis=1,inplace=True)

## Create Percentages for CPS 2020 Data

In [156]:
with open('../pickled/cps_20_data.pickle', "rb") as input_file:
    df_cps_20 = pickle.load(input_file) 

### Education Columns

In [157]:
df_cps_20['Percent_hs_grad'] = df_cps_20['pop_hs_grad'] / df_cps_20['Num_respondants_b']
df_cps_20['Percent_bachelors'] = df_cps_20['pop_bachelors'] / df_cps_20['Num_respondants_b']
df_cps_20['Percent_grad_degree'] = df_cps_20['pop_grad_degree'] / df_cps_20['Num_respondants_b']

### Citizenship Status Columns

In [158]:
df_cps_20['Percent_non_citizen'] = df_cps_20['pop_non_citizen'] / df_cps_20['Num_respondants_b']

### Disability Status Columns

In [159]:
df_cps_20['Percent_disabled'] = df_cps_20['HH_disabled']/ (df_cps_20['HH_disabled']+df_cps_20['HH_not_disabled'] )


### Health Insurance columns

In [160]:
df_cps_20['Percent_no_health'] = (df_cps_20['HH_not_insured'])/(df_cps_20['HH_health_insured']+df_cps_20['HH_not_insured']+df_cps_20['HH_some_insured'])
df_cps_20['Percent_priv_health'] = (df_cps_20['HH_health_priv']+df_cps_20['HH_some_insured_priv']) /(df_cps_20['HH_not_insured_priv'] + df_cps_20['HH_some_insured_priv']+df_cps_20['HH_health_priv'])
df_cps_20['Percent_public_health'] = (df_cps_20['HH_insured_pub']+df_cps_20['HH_some_insured_pub']) /(df_cps_20['HH_no_health_pub'] + df_cps_20['HH_some_insured_pub']+df_cps_20['HH_insured_pub'])


In [161]:
df_cps_20 = df_cps_20.loc[:,['Year','HH_income','HH_size', 'Percent_hs_grad','Percent_bachelors','Percent_grad_degree',
    'Percent_non_citizen','Percent_disabled','Percent_no_health','Percent_priv_health','Percent_public_health']].reset_index()

df_cps_20.rename(columns={'index':'FIPS', 'HH_income':'hh_med_income','HH_size':'hh_avg_size' },inplace=True)

### Concatenate 2020 CPS data with main df

In [162]:
df_no_20 = df[~df.Year.isin(['2020'])]
df_20 = df[df.Year=='2020']

df_20_dropped = df_20.drop(df_cps_20.columns[1:],axis=1)
df_20 = df_cps_20.merge(df_20_dropped, on='FIPS', how='outer')
df_20['Year'] = '2020'
df = pd.concat([df_no_20, df_20])

In [163]:
(df['Total_workforce']/df['TOT_POP']).sample(10)

1664     0.547464
1471     0.515431
39228    0.497758
1547     0.444712
2294          NaN
36048    0.435865
688      0.506522
32779    0.444195
1402     0.517265
717      0.488204
dtype: float64

# Impute Missing 2020 Data
2020 Data is only available for 329 Counties, as the CPS surveys are only performed on a subset of the population. The following cells use survey responses to impute missing values for all of the other counties, by taking an average rate of change.

Method: 
- retrieve fips for counties with missing values
- determine average change from 2019-2020 for CPS columns
- apply average rate of change to missing counties

### Determine which Counties have missing 2020 data

In [164]:
missing_fips = df_20['FIPS'][df_20.hh_med_income.isnull()].tolist()
not_missing = df_20['FIPS'][~df_20.hh_med_income.isnull()].tolist()
cps_columns = df_cps_20.columns


### Calculate mean values and percent change for 2019 & 2020

In [165]:
df_19_cps = df[cps_columns][df.Year == '2018']
means_19_20 = df_19_cps[df_19_cps['FIPS'].isin(not_missing)].mean().reset_index().rename(columns={0:'2018 Mean'})

df_20_cps = df[cps_columns][df.Year == '2020']
means_19_20['2020 Mean'] = df_20_cps[df_20_cps['FIPS'].isin(not_missing)].mean().values      
means_19_20[['2018 Mean', '2020 Mean']].pct_change(axis=1)
means_19_20['pct_change'] = (means_19_20['2020 Mean']-means_19_20['2018 Mean'])/means_19_20['2018 Mean']
means_19_20

,index,2018 Mean,2020 Mean,pct_change
0,FIPS,inf,inf,NaN
1,Year,inf,inf,NaN
2,hh_med_income,6.578080e+04,6.872134e+04,0.044702
3,hh_avg_size,2.638914e+00,1.762491e+00,-0.332115
4,Percent_hs_grad,1.828338e-01,3.861897e-01,1.112245
5,Percent_bachelors,1.323612e-01,1.450319e-01,0.095729
6,Percent_grad_degree,8.363571e-02,8.734611e-02,0.044364
7,Percent_non_citizen,5.780223e-02,4.038632e-02,-0.301302
8,Percent_disabled,1.201580e-01,2.024197e-02,-0.831539
9,Percent_no_health,8.050757e-02,3.898461e-02,-0.515765


### Apply percent change to generate missing values for 2020
Percent change rate is applied to the 2019 values of counties that are missing values for 2020.

In [166]:
df_19_20 = df_19_cps[df_19_cps.FIPS.isin(missing_fips)].drop('Year',axis=1)
df_19_20['hh_med_income'] = df_19_20['hh_med_income']+ df_19_20['hh_med_income']* 0.044702
df_19_20['hh_avg_size'] = df_19_20['hh_avg_size']+ df_19_20['hh_avg_size']* -0.332115
df_19_20['Percent_hs_grad'] = df_19_20['Percent_hs_grad']+ df_19_20['Percent_hs_grad']* 1.112245
df_19_20['Percent_bachelors'] = df_19_20['Percent_bachelors']+ df_19_20['Percent_bachelors']* 0.095729
df_19_20['Percent_grad_degree'] = df_19_20['Percent_grad_degree']+ df_19_20['Percent_grad_degree']* 0.044364
df_19_20['Percent_non_citizen'] = df_19_20['Percent_non_citizen']+ df_19_20['Percent_non_citizen']* -0.301302
df_19_20['Percent_disabled'] = df_19_20['Percent_disabled']+ df_19_20['Percent_disabled']* -0.831539
df_19_20['Percent_no_health'] = df_19_20['Percent_no_health']+ df_19_20['Percent_no_health']* -0.515765
df_19_20['Percent_priv_health'] = df_19_20['Percent_priv_health']+ df_19_20['Percent_priv_health']* 0.098888
df_19_20['Percent_public_health'] = df_19_20['Percent_public_health']+ df_19_20['Percent_public_health']* 0.432571
df_19_20

,FIPS,hh_med_income,hh_avg_size,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_non_citizen,Percent_disabled,Percent_no_health,Percent_priv_health,Percent_public_health
1428,01073,54302.565258,1.649676,0.384496,0.147515,0.089774,0.018711,0.025895,0.045649,0.737636,0.496157
1429,01117,79147.668222,1.776574,0.285176,0.203412,0.098840,0.023183,0.018425,0.032084,0.864214,0.337877
1430,04003,50823.707598,1.589566,0.343544,0.108584,0.062698,0.034888,0.026485,0.036652,0.639360,0.661645
1435,05119,52332.257286,1.642997,0.375347,0.155338,0.093706,0.024338,0.025749,0.041235,0.707271,0.556736
1437,06013,97901.113824,1.910151,0.250803,0.193429,0.110050,0.079108,0.018558,0.027016,0.802932,0.442811
...,...,...,...,...,...,...,...,...,...,...,...
52741,05089,37105.725636,1.629639,0.580883,0.088763,0.050009,0.003648,0.037796,0.033038,0.617263,0.796962
52742,36035,52494.186096,1.562851,0.543293,0.074774,0.062175,0.003622,0.026447,0.030156,0.698368,0.652193
52743,20209,46878.912846,1.816647,0.431633,0.079442,0.038295,0.089741,0.024463,0.088502,0.575504,0.532072
52744,22057,55462.184478,1.763216,0.573715,0.084443,0.036421,0.014661,0.027483,0.048675,0.718049,0.513574


In [167]:
df_no_20 = df[~df.Year.isin(['2020'])]
df_20 = df[df.Year=='2020']

In [168]:
df_20_not_missing_cps = df_20[cps_columns][df_20.FIPS.isin(not_missing)].drop('Year',axis=1)
df_20_cps_all_fips = pd.concat([df_19_20, df_20_not_missing_cps])
df_20_cps_all_fips

,FIPS,hh_med_income,hh_avg_size,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_non_citizen,Percent_disabled,Percent_no_health,Percent_priv_health,Percent_public_health
1428,01073,54302.565258,1.649676,0.384496,0.147515,0.089774,0.018711,0.025895,0.045649,0.737636,0.496157
1429,01117,79147.668222,1.776574,0.285176,0.203412,0.098840,0.023183,0.018425,0.032084,0.864214,0.337877
1430,04003,50823.707598,1.589566,0.343544,0.108584,0.062698,0.034888,0.026485,0.036652,0.639360,0.661645
1435,05119,52332.257286,1.642997,0.375347,0.155338,0.093706,0.024338,0.025749,0.041235,0.707271,0.556736
1437,06013,97901.113824,1.910151,0.250803,0.193429,0.110050,0.079108,0.018558,0.027016,0.802932,0.442811
...,...,...,...,...,...,...,...,...,...,...,...
324,55073,47870.090909,1.848485,0.542857,0.171429,0.000000,0.000000,0.000000,0.090909,0.681818,0.545455
325,55101,97732.240000,2.240000,0.512195,0.146341,0.024390,0.024390,0.025641,0.000000,0.846154,0.461538
326,55105,55234.312500,1.875000,0.533333,0.116667,0.083333,0.033333,0.041667,0.000000,0.625000,0.625000
327,55139,75513.866667,1.700000,0.421875,0.203125,0.062500,0.000000,0.000000,0.000000,0.650000,0.550000


In [169]:
not_cps_columns = list(set(df_20.columns)-set(cps_columns[1:]))
df_20_no_cps = df_20[not_cps_columns]
df_20_new = df_20_cps_all_fips.merge(df_20_no_cps,on='FIPS',how='outer')
df_20_new['Year']='2020'
df = pd.concat([df_no_20,df_20_new])

## Impute Missing 2020 Demographic Data
Some features, such as demographic information, follow a predictable pattern and are unlikely to be sharply affected by the pandemic, therefore these columns will be imputed using previous yearly data to calculate estimated 2020 values.
### Determine yearly percent change for demographics columns

In [170]:
df.groupby('Year')[['Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC','Percent_disabled', 'Percent_hs_grad',
     'Percent_bachelors','Percent_grad_degree','Percent_priv_health','Percent_public_health','Percent_no_health']].mean()


,Percent_male,Percent_female,Percent_white,Percent_Black,Percent_native,Percent_asian,Percent_pacific,Percent_latinX,Percent_PoC,Percent_disabled,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_priv_health,Percent_public_health,Percent_no_health
Year,,,,,,,,,,,,,,,,
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,0.499648,0.500352,0.858055,0.090013,0.021626,0.012630,0.000961,0.084257,0.141945,NaN,0.238260,0.084332,0.043167,NaN,NaN,NaN
2011,0.499541,0.500459,0.839700,0.110203,0.018216,0.013882,0.001130,0.064932,0.160300,NaN,0.237480,0.084378,0.043308,NaN,NaN,NaN
2012,0.499773,0.500227,0.838104,0.110548,0.018413,0.014356,0.001165,0.066211,0.161896,0.149516,0.236568,0.085763,0.044233,0.627519,0.331782,0.146020
2013,0.499853,0.500147,0.835338,0.111947,0.018647,0.014926,0.001204,0.067619,0.164662,0.150693,0.237064,0.087005,0.045306,0.619998,0.339705,0.146622
2014,0.500625,0.499375,0.850215,0.092106,0.022659,0.014644,0.001266,0.091117,0.149785,0.150614,0.237156,0.090137,0.047232,0.625153,0.341686,0.140750
2015,0.500818,0.499182,0.848638,0.092428,0.022878,0.015110,0.001301,0.092774,0.151362,0.151874,0.237364,0.091465,0.048343,0.628070,0.350993,0.130266
2016,0.500852,0.499148,0.847006,0.092782,0.023124,0.015556,0.001332,0.094446,0.152994,0.153260,0.236615,0.093128,0.049572,0.631643,0.360194,0.118918
2017,0.500875,0.499125,0.845457,0.093154,0.023354,0.015933,0.001368,0.096121,0.154543,0.153586,0.235818,0.094946,0.050888,0.635518,0.367423,0.107984


In [171]:
percent_change = df.groupby('Year')[['Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC']].mean().pct_change().mean()
percent_change


Percent_male       0.000192
Percent_female    -0.000192
Percent_white     -0.002104
Percent_Black      0.008908
Percent_native     0.011271
Percent_asian      0.041518
Percent_pacific    0.033430
Percent_latinX     0.028241
Percent_PoC        0.013349
dtype: float64

### Multiply each 2019 value by the calculated percent change by above, to derive a 2020 estimate

In [172]:
dems_19_20 = df[['FIPS','Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC']][df.Year=='2019'].drop_duplicates()

dems_19_20['Percent_male'] = dems_19_20['Percent_male'] + dems_19_20['Percent_male']* 0.000192
dems_19_20['Percent_female'] = dems_19_20['Percent_female'] + dems_19_20['Percent_female']* -0.000192
dems_19_20['Percent_white'] = dems_19_20['Percent_white']+ dems_19_20['Percent_white']* -0.002104
dems_19_20['Percent_Black'] = dems_19_20['Percent_Black'] +dems_19_20['Percent_Black']* 0.008908
dems_19_20['Percent_native'] = dems_19_20['Percent_native']+ dems_19_20['Percent_native']* 0.011271
dems_19_20['Percent_asian'] = dems_19_20['Percent_asian'] +dems_19_20['Percent_asian']* 0.041518
dems_19_20['Percent_pacific'] = dems_19_20['Percent_pacific'] +dems_19_20['Percent_pacific']* 0.033430
dems_19_20['Percent_latinX'] = dems_19_20['Percent_latinX'] +dems_19_20['Percent_latinX']* 0.028241
dems_19_20['Percent_PoC'] = dems_19_20['Percent_PoC'] + dems_19_20['Percent_PoC']* 0.013349


### Drop nulls, and add the newly calculated 2020 values into the main df

In [173]:
df_20_dropped_null = df[df.Year=='2020'].drop(['Percent_male','Percent_female','Percent_white','Percent_Black',
            'Percent_native','Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC'],axis=1)
df_not_20 = df[df.Year!='2020']
df_20_new = df_20_dropped_null.merge(dems_19_20,on='FIPS',how='outer')

df = pd.concat([df_not_20, df_20_new])

## Impute Houselessness Data
*NOTE* Decided not to use imputed 2020 houselessness data, because it created significant null values.<br>
Data is still widely unavailable for 2020 houselessness rates, however estimates are available at the City level in some areas. Greater Los Angeles is [experiencing a 10% increase](https://www.lahsa.org/documents?id=4585-2020-greater-los-angeles-homeless-count-los-angeles-continuum-of-care-coc-) in houseless rates for 2020, therefore this rate will be applied to each county's 2019 houseless rate, to create an estimate for 2020.

In [174]:
# houseless_19_20 = df[['FIPS','Houseless_rate']][df.Year=='2019'].drop_duplicates()
# houseless_19_20['Houseless_rate'] = houseless_19_20['Houseless_rate']*1.13


In [175]:
# df_not_20 = df[df.Year!='2020']
# df_20_dropped_houseless = df[df.Year=='2020'].drop(['Houseless_rate'],axis=1)
# df_20_new = df_20_dropped_houseless.merge(houseless_19_20,on='FIPS',how='outer')
# df = pd.concat([df_not_20, df_20_new])
# df

# Polynomial Features
**Note:** Decided to omit polynomial features, because they decreased model performance.

In [176]:
# only using derived percentages, ignore raw counts
continuous_features = ['Rent', 'Houseless_rate','Sheltered_rate', 'Unsheltered_rate', 'TOT_POP',
       'Cost Per Meal', 'Num_wholesale','Num_restaraunts', 'Num_grocery',  'Unemployment_rate', 'Percent_male', 
         'Percent_female','Percent_white', 'Percent_Black', 'Percent_native', 'Percent_asian',
       'Percent_pacific', 'Percent_latinX','Percent_working', 'Total_food_retail','Food_retail_per_person',
        'Percent_disabled','Percent_hs_grad','Percent_bachelors','Percent_grad_degree','Percent_priv_health',
            'Percent_public_health','Percent_no_health','Percent_65+','Percent_non_citizen','Percent_hh_no_vehicle',
                    'Percent_hh_SNAP','percent_hh_poverty','hh_avg_size','num_hh','hh_med_income']


In [177]:
# # add squared and cubed polynomials for each continuous feature
# for feat in continuous_features:
#     df[feat+'^2'] = df[feat]**2
#     df[feat+'^3'] = df[feat]**3


# Interaction Features
Create an interaction feature for each combination of continuous features, and add best ones to dataframe

In [178]:
# Generate combinations of features
y = df.dropna()[['FI Rate']]
X = df.dropna()[continuous_features]
interactions = list(combinations(X.columns, 2))
interaction_dict = {}

# run simple regression model with each possible interaction, and save R-squared for each interaction in a dictionary
for interaction in interactions:
    X_copy = X.copy()
    X_copy['interact'] = X_copy[interaction[0]] * X_copy[interaction[1]] 
    X_copy = X_copy.replace([np.inf, -np.inf], 0)
    model = LinearRegression()
    model.fit(X_copy, y)
    interaction_dict[model.score(X_copy, y)] = interaction 


### Add best 50 interactions to dataframe

In [179]:
# Sort the interactions dictionary, and add best 50 interactions to dataframe
top_interactions = sorted(interaction_dict.keys(), reverse = True)[:50]
for interaction in top_interactions:
    feature1 = interaction_dict[interaction][0]
    feature2 = interaction_dict[interaction][1]
    df[feature1+'_X_'+feature2] = df[feature1] * df[feature2] #also add to new_features df


# Log Transformations
Take natural log of each continuous feature, and add these log features to dataframe

In [180]:
for feat in continuous_features:
    df['log_'+feat] = df[feat].map(lambda x: np.log(x))
df = df.replace([np.inf, -np.inf], 0)

# Dummy Variables
High and Low Threshold programs delineate the assistance programs provided by State

In [181]:
# Create dummy variables for high and low threshold programs, and add to dataframe
hi_thresh_dummies = pd.get_dummies(df['High Threshold Type'].astype(str), dtype=int)
hi_thresh_dummies['other'] = hi_thresh_dummies['Other Nutrition Program'] + hi_thresh_dummies['other nutrition pgm']
hi_thresh_dummies.drop(['Other Nutrition Program','other nutrition pgm','nan'],axis=1,inplace=True) # drop last col
hi_thresh_dummies.rename(columns = {'SNAP, Other Nutrition Programs': 'SNAP_other'}, inplace=True)
hi_thresh_dummies.columns = 'Hi_thresh_'+hi_thresh_dummies.columns

low_thresh_dummies = pd.get_dummies(df['Low Threshold Type'].astype(str), dtype=int)
low_thresh_dummies.drop('nan', axis=1,inplace=True)
low_thresh_dummies.rename(columns = {'SNAP, Other Nutrition Programs': 'SNAP_other'}, inplace=True)
low_thresh_dummies.columns = 'Lo_thresh_'+low_thresh_dummies.columns

df = pd.concat([df, low_thresh_dummies, hi_thresh_dummies],axis=1)


### Pickle the new dataframe
Save the dataframe with all new features added (demographic percentages, interactions, logs, and dummies)

In [182]:
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,percent_hh_poverty,hh_avg_size,num_hh,State,County,TOT_POP,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate,Percent_male,Percent_female,Percent_white,Percent_Black,Percent_native,Percent_asian,Percent_pacific,Percent_latinX,Percent_PoC,Percent_working,Total_food_retail,Food_retail_per_person,Percent_disabled,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_priv_health,Percent_public_health,Percent_no_health,Percent_65+,Percent_non_citizen,Percent_hh_no_vehicle,Percent_hh_SNAP,Unemployment_rate_X_Percent_Black,Unemployment_rate_X_Percent_white,Unemployment_rate_X_Percent_latinX,Houseless_rate_X_Sheltered_rate,Sheltered_rate_X_Percent_grad_degree,Percent_Black_X_Percent_hh_SNAP,Percent_latinX_X_hh_med_income,Cost Per Meal_X_Percent_Black,Unemployment_rate_X_hh_avg_size,Percent_white_X_Percent_hh_SNAP,Rent_X_Unemployment_rate,Percent_Black_X_Percent_grad_degree,Percent_pacific_X_Percent_hs_grad,Percent_latinX_X_Percent_hh_SNAP,Unemployment_rate_X_Percent_non_citizen,Percent_white_X_Percent_grad_degree,Percent_latinX_X_percent_hh_poverty,TOT_POP_X_Percent_latinX,Unemployment_rate_X_Percent_female,Unemployment_rate_X_Percent_male,Percent_pacific_X_Percent_public_health,Percent_Black_X_Percent_hs_grad,Percent_latinX_X_num_hh,Rent_X_Percent_Black,Num_grocery_X_num_hh,Houseless_rate_X_Percent_pacific,Percent_white_X_percent_hh_poverty,Percent_white_X_Percent_priv_health,Percent_white_X_Percent_pacific,Unemployment_rate_X_Percent_hs_grad,Unemployment_rate_X_Percent_disabled,Percent_white_X_Percent_no_health,Percent_female_X_Percent_hs_grad,Sheltered_rate_X_Percent_working,Percent_Black_X_Percent_no_health,Percent_Black_X_percent_hh_poverty,Percent_pacific_X_Percent_disabled,Cost Per Meal_X_Percent_white,Percent_pacific_X_Percent_65+,Sheltered_rate_X_Percent_no_health,TOT_POP_X_Num_grocery,Percent_Black_X_hh_med_income,Percent_Black_X_Percent_priv_health,Percent_hs_grad_X_hh_avg_size,Sheltered_rate_X_Percent_pacific,Unemployment_rate_X_Percent_hh_SNAP,Percent_latinX_X_Percent_priv_health,Total_food_retail_X_num_hh,Houseless_rate_X_Percent_grad_degree,Percent_pacific_X_Percent_hh_SNAP,log_Rent,log_Houseless_rate,log_Sheltered_rate,log_Unsheltered_rate,log_TOT_POP,log_Cost Per Meal,log_Num_wholesale,log_Num_restaraunts,log_Num_grocery,log_Unemployment_rate,log_Percent_male,log_Percent_female,log_Percent_white,log_Percent_Black,log_Percent_native,log_Percent_asian,log_Percent_pacific,log_Percent_latinX,log_Percent_working,log_Total_food_retail,log_Food_retail_per_person,log_Percent_disabled,log_Percent_hs_grad,log_Percent_bachelors,log_Percent_grad_degree,log_Percent_priv_health,log_Percent_public_health,log_Percent_no_health,log_Percent_65+,log_Percent_non_citizen,log_Percent_hh_no_vehicle,log_Percent_hh_SNAP,log_percent_hh_poverty,log_hh_avg_size,log_num_hh,log_hh_med_income,Lo_thresh_SNAP,Lo_thresh_SNAP_other,Hi_thresh_SNAP,Hi_thresh_SNAP_other,Hi_thresh_other
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.000000,14.8,2.480000,259397.0,Alabama,Jefferson County,659972.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.0,292505.0,19626.0,6.30,0.473491,0.526509,0.538864,0.430446,0.003074,0.015725,0.000526,0.036515,0.461136,0.472946,3576.0,0.005418,0.143315,0.178574,0.123681,0.079964,0.654025,0.320574,0.123242,0.131878,0.026545,0.360953,0.154077,2.711807,3.394842,0.230046,0.000001,0.000077,0.066322,1651.910477,1.261205,15.624000,0.083026,6425.974545,0.034420,0.000094,0.005626,0.167234,0.043090,0.540425,24099.0,3.317005,2.982995,0.000169,0.076866,9471.929571,439.052706,1.037588e+08,7.338020e-07,7.975184,0.352430,0.000283,1.125018,0.902886,0.066410,0.094021,0.000454,0.053049,6.370594,0.000075,1.578871,0.000069,0.000118,2.639888e+08,19472.925515,0.281522,0.442864,5.052

In [183]:
with open('../pickled/feature_engineered_data.pickle', "wb") as output_file:
    pickle.dump(df, output_file)